In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
%matplotlib tk

In [9]:
from renewableopt.data import load_by_day
import matplotlib.pyplot as plt
import numpy as np
from numpy.fft import fft, fftshift

In [4]:
t, load, gen_pu = load_by_day(0, 366, sources=['solar'])

In [56]:
# Make length(gen_pu) a multiple of N
def clip_multiple(x, N):
    r = int(np.floor(len(x) / N))
    return x[:N * r]
    
def offset_slice(s, x):
    return slice(s.start + x, s.stop + x)
    
def periodogram(x, N, overlap, window_func):
    x = clip_multiple(x, N)
    L = np.shape(x)[0]
    M = np.floor((L-overlap) / (N - overlap))
    w = window_func(N)
    S = np.zeros(N)
    index = slice(0, N)
    for i in range(N-1):
        if index.stop > L:
            break
        S += np.abs(fft(x[index] * w))**2
        index = offset_slice(index, N - overlap)
    return fftshift(S) / (M * N**2)

    
    

In [58]:
N = 4096
t = clip_multiple(t, N)
gen_pu = clip_multiple(gen_pu, N)
S = periodogram(gen_pu, N, int(N/4), np.blackman)

In [66]:
delta_t = t[1] - t[0]
assert np.all(np.isclose(t[1:] - t[:-1], delta_t))

In [70]:
delta_f = 1 / (N * delta_t)
freq = delta_f * (np.arange(N) - N//2)

In [81]:
plt.figure()
plt.semilogy(24 * 60 * freq, S)
plt.xlabel("Frequency (1 / day)")
plt.ylabel("Spectral density")


Text(0, 0.5, 'Spectral density')

In [77]:
max(freq)

0.09995117187500001

In [79]:
delta_f * 24 * 60

0.07031250000000001

In [96]:
def autocorrelate(x, mode='full'):
    # return np.correlate(gen_pu, gen_pu, mode='full')
    L = len(x)
    Rx = np.zeros_like(x)
    for n in range(L):
        Rx[n] = np.dot(x[:L-n], x[n:]) / (L - n)
    return Rx

In [101]:
Rxx = autocorrelate(gen_pu)

In [ ]:
Rx = aut

In [84]:
Rxx.shape

(204799,)

In [99]:
plt.figure()
plt.plot(np.arange(len(Rxx)) * delta_t / 24 / 60, Rxx)

In [100]:
plt.figure()
plt.plot(fft(Rxx))

/home/sean/code/renewableopt/.hatch/jupyter/lib/python3.8/site-packages/matplotlib/cbook/__init__.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [80]:
1 / delta_f / 24 / 60

14.222222222222223

In [67]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [64]:
min(t[1:] - t[:-1])

5.0

In [61]:
np.isclose(1)

TypeError: isclose() missing 1 required positional argument: 'b'

In [46]:
plt.plot(S)

In [33]:
N

2635

In [23]:
a = slice(10, 15)

In [12]:
gen_pu[a]

array([0., 0., 0., 0., 0.])

In [16]:
a.stop

15